# Outfit Compatibility Prediction

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
from collections import Counter
from PIL import Image
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import pickle

import sys
sys.path.insert(0, "/recsys_data/RecSys/fashion/automl/efficientnetv2")
import effnetv2_model

%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [3]:
base_dir = "/recsys_data/RecSys/fashion/polyvore-dataset/polyvore_outfits"
train_dir = os.path.join(base_dir, "disjoint")
image_dir = os.path.join(base_dir, "images")
train_json = "train.json"
train_file = "compatibility_train.txt"
item_file = "polyvore_item_metadata.json"
outfit_file = "polyvore_outfit_titles.json"

In [4]:
with open(os.path.join(train_dir, train_json), 'r') as fr:
    pv_train_pos = json.load(fr)
    
with open(os.path.join(base_dir, item_file), 'r') as fr:
    pv_items = json.load(fr)
    
with open(os.path.join(base_dir, outfit_file), 'r') as fr:
    pv_outfits = json.load(fr)
    
with open(os.path.join(train_dir, train_file), 'r') as fr:
    train_X, train_y = [], []
    for line in fr:
        elems = line.strip().split()
        train_y.append(elems[0])
        train_X.append(elems[1:])


In [5]:
item_dict = {}
for ii, outfit in enumerate(pv_train_pos):
    items = outfit['items']
    mapped = train_X[ii]
    item_dict.update({jj:kk['item_id'] for jj, kk in zip(mapped, items)})

In [ ]:
import glob
efficient_net_dict = {}

eff2_model = tf.keras.models.Sequential(
                [
                    tf.keras.layers.InputLayer(input_shape=[224, 224, 3]),
                    effnetv2_model.get_model("efficientnetv2-b0", include_top=False),
                ]
            )

for image_path in tqdm(glob.glob(image_dir+"/*.jpg")):
    item_id = image_path.split('/')[-1].split('.')[0]
    image = tf.keras.preprocessing.image.load_img(image_path)
    image_arr = tf.keras.preprocessing.image.img_to_array(image)
    image_arr = tf.image.resize(image_arr, (224, 224)).numpy()
    image_arr /= 255.0
    image_embed = tf.squeeze(eff2_model(tf.expand_dims(image_arr, 0)))
    efficient_net_dict[item_id] = image_embed

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.
 70%|███████   | 183608/261057 [5:51:32<2:25:03,  8.90it/s]

In [ ]:
print("Here")

In [7]:
with open("effnet2_polyvore.pkl", "wb") as output_file:
    pickle.dump(efficient_net_dict, output_file)

TensorShape([1280])

In [6]:
from data_process import CustomDataGen

In [7]:
traingen = CustomDataGen(train_X, train_y, item_dict, pv_items, image_dir, batch_size=32)
x, y = traingen[0]
x.shape, y.shape

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


((32, 16, 1280), (32,))

In [1]:
from build_model import build_multilevel_transformer

In [2]:
model = build_multilevel_transformer(16, (224, 224, 3))

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


AssertionError: in user code:

    /recsys_data/RecSys/fashion/transformer_models_ts.py:277 call  *
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    /anaconda/envs/py37_tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:897 __call__  **
        self._maybe_build(inputs)
    /anaconda/envs/py37_tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:2416 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    /anaconda/envs/py37_tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/layers/wrappers.py:183 build
        super(TimeDistributed, self).build(tuple(child_input_shape))
    /anaconda/envs/py37_tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/layers/wrappers.py:58 build
        self.layer.build(input_shape)
    /recsys_data/RecSys/fashion/transformer_models_ts.py:19 build
        effnetv2_model.get_model("efficientnetv2-b0", include_top=False),
   

In [2]:
class image_embedding_layer(tf.keras.layers.Layer):
    def __init__(self, dropout, embedding_dim, activation):
        super(image_embedding_layer, self).__init__()
        self.dropout = dropout
        self.embedding_dim = embedding_dim
        self.activation = activation

    def build(self, input_shape):
        self.layer = tf.keras.Sequential(
            [
                effnetv2_model.get_model("efficientnetv2-b0", include_top=False),
                tf.keras.layers.Dropout(rate=self.dropout),
                tf.keras.layers.Dense(self.embedding_dim, activation=self.activation),
            ]
        )

    def call(self, input_tensor):
        return self.layer(input_tensor)


In [3]:
inputs = tf.keras.layers.Input(shape=[224, 224, 3])
output = image_embedding_layer(0.1, 256, "tanh")(inputs)
model = tensorflow.keras.models.Model(inputs=inputs, outputs=output)
model.summary()

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


AssertionError: Some objects had attributes which were not restored:
    <tf.Variable 'image_embedding_layer/efficientnetv2-b0/stem/conv2d/kernel:0' shape=(3, 3, 3, 32) dtype=float32, numpy=
array([[[[ 2.08487988e-01,  9.88112614e-02, -9.59084034e-02,
          -6.33243769e-02,  8.29396956e-03, -5.07830235e-04,
          -5.39753549e-02, -1.39424890e-01,  6.77010790e-02,
          -1.93767026e-02, -1.08512893e-01,  4.35398631e-02,
          -6.34635538e-02, -8.76769274e-02, -1.63157523e-01,
           3.15265879e-02, -5.62494956e-02, -1.03921264e-01,
          -6.72708591e-03, -3.95094119e-02, -1.58124015e-01,
           5.32882325e-02, -2.84029059e-02,  9.30022523e-02,
           1.67917237e-02, -1.07609294e-01,  4.07438986e-02,
           6.89973012e-02, -1.58118784e-01,  9.06480402e-02,
           1.04643606e-01, -2.64211185e-02],
         [-5.08350506e-03, -4.75224704e-02, -2.30943486e-02,
           2.30589546e-02, -9.13527608e-03,  2.67817127e-03,
           9.76234078e-02,  2.62

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[224, 224, 3]),
    effnetv2_model.get_model('efficientnetv2-b0', include_top=False),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(4, activation='softmax'),
])

In [9]:
im_layer = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[224, 224, 3]),
    effnetv2_model.get_model('efficientnetv2-b0', include_top=False),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(4, activation='softmax'),
])

AssertionError: Some objects had attributes which were not restored:
    <tf.Variable 'efficientnetv2-b0_3/stem_2/conv2d/kernel:0' shape=(3, 3, 3, 32) dtype=float32, numpy=
array([[[[ 1.61402337e-02, -1.68192089e-01, -3.92452814e-02,
          -2.54823025e-02,  5.25467619e-02, -8.73622745e-02,
           1.13614559e-01, -1.64555311e-01, -1.52179778e-01,
          -7.95886219e-02,  1.17705509e-01,  1.36144459e-01,
          -1.26637027e-01,  5.03572337e-02, -3.66919450e-02,
           3.33240405e-02, -3.35822403e-02, -1.63472235e-01,
          -3.25794592e-02,  6.27040267e-02,  5.88674098e-02,
           1.06686708e-02,  9.33022127e-02, -1.21090166e-01,
           3.51961441e-02, -7.58656766e-03, -3.20767164e-02,
          -6.22840039e-02, -5.28552532e-02, -8.66008997e-02,
          -7.30775576e-03,  1.30739305e-02],
         [ 4.03160825e-02,  8.65824074e-02, -1.08480593e-03,
           7.22659156e-02,  1.31560564e-01, -1.02846496e-01,
           3.84937711e-02, -2.76794583e-02, -2.063

In [6]:
inputs = tf.keras.Input(shape=(16, 224, 224, 3))
outputs = tf.keras.layers.TimeDistributed(model)(inputs)
outputs.shape

NotImplementedError: 

In [10]:
MODEL = 'efficientnetv2-b0'  #@param

def download(m):
    if m not in os.listdir():
        !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/{m}.tgz
        !tar zxf {m}.tgz
    ckpt_path = os.path.join(os.getcwd(), m)
    return ckpt_path

# Download checkpoint.
ckpt_path = download(MODEL)
if tf.io.gfile.isdir(ckpt_path):
    ckpt_path = tf.train.latest_checkpoint(ckpt_path)

--2022-07-09 07:51:48--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/efficientnetv2-b0.tgz
Resolving storage.googleapis.com... 142.250.138.128, 142.251.45.48, 142.251.35.208, ...
Connecting to storage.googleapis.com|142.250.138.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105641132 (101M) [application/octet-stream]
Saving to: ‘efficientnetv2-b0.tgz’

efficientnetv2-b0.t 100%[===================>] 100.75M  43.7MB/s    in 2.3s    

2022-07-09 07:51:50 (43.7 MB/s) - ‘efficientnetv2-b0.tgz’ saved [105641132/105641132]

